In [55]:
import os
import tensorflow as tf
#import keras
from keras.layers import Add,Multiply,Softmax,Input,TimeDistributed,Dense,Average,GlobalAveragePooling1D,Concatenate,Lambda,RepeatVector, Conv2D,ConvLSTM2D, MaxPooling2D,BatchNormalization,Flatten,Reshape,UpSampling2D
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import pylab as pl
from IPython import display
from IPython.core.display import HTML
from IPython.core.display import display as html_width
html_width(HTML("<style>.container { width:90% !important; }</style>"))
import tensorflow_probability as tfp
import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec
import imageio
from tqdm import tqdm
import cv2
from tensorflow.keras.layers import Activation, BatchNormalization, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import psutil
import pickle

import PIL.Image as Image

import gc
from tensorflow.keras import backend as k

# Import or redefine the custom loss function before loading
@tf.keras.utils.register_keras_serializable()
def custom_loss(y_true, y_predicted):
    mean, log_sigma = tf.split(y_predicted, 2, axis=-1)
    y_true_value, _ = tf.split(y_true, 2, axis=-1)
    sigma = tf.nn.softplus(log_sigma)
    dist = tfp.distributions.MultivariateNormalDiag(loc=mean, scale_diag=sigma)
    loss = -tf.reduce_mean(dist.log_prob(y_true_value))
    return loss

# Load the model with the registered loss function
model = keras.models.load_model("Epochs_V7/Epoch138000/model.keras", custom_objects={"custom_loss": custom_loss})

# Print model summary to verify it loaded correctly
model.summary()

/tmp/ipykernel_2805913/265494496.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display as html_width


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_observation (InputLa  [(None, None, 128, 128, 4)   0         []                            
 yer)                        ]                                                                    
                                                                                                  
 time_distributed_7 (TimeDi  (None, None, 128, 128, 32)   1184      ['image_observation[0][0]']   
 stributed)                                                                                       
                                                                                                  
 time_distributed_8 (TimeDi  (None, None, 64, 64, 32)     0         ['time_distributed_7[0][0]']  
 stributed)                                                                                   

In [56]:
def get_data(t,n, size):
    d = t
    out = []
    #print(d)
    coef = 1
    img_coef = np.ones((1, 128)) * coef
    touch_coef = np.ones((1, 128)) * (1 - coef)

    observation = np.zeros((1, n, 128, 128, 4)) 
    observation_touch = np.zeros((1, n, 9)) 
    target_X = np.zeros((1, 1))

    pose = np.loadtxt('Data_Robot/%d/touch_data.txt' % d)
    time_len = pose.shape[0]
    times = np.linspace(0, 1, time_len)

    target_Y = np.zeros((1, 128, 128, 6))
    target_Y_touch = np.zeros((1, 8))

    for idx in range(size):
        if idx < n:
            observation[0, idx, :, :, 0] = np.ones((128, 128)) * times[idx]
            image = mpimg.imread('Data_Robot/%d/frame_%d.jpeg' % (d, idx))
            image_resized = cv2.resize(image, (128, 128))
            observation[0, idx, :, :, 1:] = image_resized 
            target_Y[0, :, :, :3] = image_resized 

            observation_touch[0, idx, 0] = times[idx]
            observation_touch[0, idx, 1:9] = pose[idx] / 255
            target_Y_touch[0, :8] = pose[idx] / 255
        else:
            t_img = mpimg.imread('Data_Robot/%d/frame_%d.jpeg' % (d, idx))
            t_img_resize = cv2.resize(t_img, (128, 128))
            target_Y[0, :, :, :3] = t_img_resize 
            target_Y_touch[0, :8] = pose[idx] / 255
        out.append([target_Y.copy(), target_Y_touch.copy()])

    target_X[0, 0] = times[n]

    return [observation, observation_touch, target_X, img_coef, touch_coef], out, idx, times

#inp, out, x, clock = get_test_sample(1,20,55)
#print(out[44][0])   


In [57]:
def classi(pred_touch_values, count):
    ignore_indices = {0, 4}
    mean = []

    for i in range(count):
        arr = pred_touch_values[i]
        mask = np.ones_like(arr, dtype=bool)
        mask[list(ignore_indices)] = False 

        filtered_arr = arr[mask]

        max_value = np.max(filtered_arr)
        mean.append(max_value)


    mean_value = np.mean(mean)

    if mean_value >= 0.75:
        print("PAIN")
    elif mean_value > 0.02:
        print("TOUCH")
    else:
        print("No Touch")

In [58]:
t_inp = 20
pred_size = 25
sample = 49

test_inp, test_out, x, times = get_data(sample, t_inp, pred_size)

step_predictions = []
step_real_values = []
count = 0

for i in range(t_inp+1, pred_size-1):
    inp_copy = test_inp.copy() 
    inp_copy[2] = np.array([[times[i]]])
    pred = model.predict(inp_copy)

    step_predictions.append(pred)
    step_real_values.append(test_out[i])

    pred_images = np.array(pred[0])
    pred_touch_values = np.array(pred[1])  

    out_images = np.array(test_out[i][0])  
    out_touch_values = np.array(test_out[i][1]) 
    count +=1

pred_touch_values = np.array([p[1].squeeze() for p in step_predictions])
real_touch_values = np.array([r[1].squeeze()for r in step_real_values])
pred_img_values = np.array([p[0].squeeze() for p in step_predictions])  
real_img_values = np.array([r[0].squeeze() for r in step_real_values])

classi(pred_touch_values, count)

# display.clear_output(wait=True)
# display.display(pl.gcf())
# plt.imshow(real_img_values[2,:,:,:3])
# plt.show()
# plt.imshow(pred_img_values [2,:,:,:3])
# plt.show()


1/1 [==============================] - 0s 18ms/step
PAIN
